# Colab을 활용한 간단한 예제 작성

## 1. Colab 서버 스펙 확인

### 전반적인 사양

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### OS 정보

In [ ]:
import platform
platform.platform()

### OS 모델

In [ ]:
  !cat /etc/issue.net

### Memory

In [ ]:
!cat /proc/meminfo

### CPU 정보

In [ ]:
! cat /proc/cpuinfo

### Disk 정보

In [ ]:
!df -h

### GPU 정보

In [ ]:
!nvidia-smi

### Python 버전 확인

In [ ]:
!python --version

### 현재 디렉토리 목록 확인

In [ ]:
!ls

## 2. 파일처리

### 파일쓰기

In [ ]:
%%writefile  test.py
print('hello world!')

In [ ]:
# 위에서 생성한 test.py가 조회된다.
!ls

### 파이썬 코드 실행

In [ ]:
# test.py 실행시키기
%run test.py

### 파일 다운로드

In [ ]:
from google.colab import files
# 브라우저에 다운로드 됨을 확인할 수 있다.
files.download('test.py')

### 파일업로드

In [ ]:
# [Cancel upload] 버튼을 클릭하여 잠시 멈춘 후 파일선택 버튼을 클릭하면 PC 내 파일을 선택할 수 있는 다이얼로그 창이 뜬다.
# 리턴값을 받는 변수인 myupload라는 이름의 디렉토리가 생성된다.
myupload = files.upload()

## 3. 구글드라이브 연동

기존에는 연동을 위한 인증절차가 꽤 복잡하여 사전에 20줄 정도되는 스크립트로 OAuth를 가동하기 위한 준비가 필요했었다.
하지만 지금은 업그레이드되어 굉장히 편리해졌다.

### 현재 디렉토리 위치 및 목록 확인

In [ ]:
import os
print(os.getcwd())
!ls

### 구글드라이브 연동

In [ ]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
# 마운트된 내 드라이브를 확인해보자
!ls

In [ ]:
# 해당 드라이브로 이동 
# 내 드라이브는 원격서버가 아니라 로컬서버로 간주하므로 명령어 실행시 앞단에 !를 붙이지 않는다.
cd MyDrive/My Drive

In [ ]:
# 내드라이브의 전체 목록이 나타난다.
ls

In [ ]:
# 특정파일을 가져오고 싶은 경우 다음과 같이 접근한다.
import pandas as pd
df = pd.read_csv("./MyDrive/test/test.csv")

## 4. 텐서플로우 실행

## 텐서플로우 샘플 CNN(컨볼루션 신경망 계층)의 성능측정(CPU vs GPU)
* GPU가 압도적으로 빠른것을 확인할 수 있다.

In [ ]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/device:GPU:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Ensure that TF can detect the GPU.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

## 5. 케라스 샘플코드 테스트

In [ ]:
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation

# mnist import
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# 0~1 사이의 값으로 정규화
X_train = X_train.reshape(60000, 784).astype('float32') / 255.0
X_test = X_test.reshape(10000, 784).astype('float32') / 255.0

# 원핫 인코딩
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

model = Sequential()
model.add(Dense(units=64, input_dim=28*28, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
hist = model.fit(X_train, Y_train, epochs=2, batch_size=32)

loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)

print('loss_and_metrics : ' + str(loss_and_metrics))

## 6. 기타

* 상단의 OS 모델에서 확인했듯이 우분투의 apt-get 명령어를 활용하여 다양한 설치가 가능하다.
* Pytorch 등이 이용가능하며 텐서플로우 또한 다른버전을 설치하여 사용할 수 있다.
* 구글 BigQuery 사용가능(pd.read_gbq 함수 활용하여 SQL실행)
* Kaggle 연동 가능
* REST API- BytesIO 연동 가능

그 외 새로운 기능들을 시간나는대로 틈틈이 추가 기술하겠습니다.